In [2]:
!pip install python-dotenv


  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)


In [1]:
import time
import smtplib
import re
import requests
from bs4 import BeautifulSoup as BS
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import win10toast
import dotenv
from dotenv import dotenv_values
import winsound

def bb():
    for _ in range(120):
        winsound.Beep(500,500)
        time.sleep(0.7)
        winsound.Beep(700,500)
        time.sleep(0.2)
        winsound.Beep(700,500)

In [3]:
import time
import smtplib
import re
import requests
from bs4 import BeautifulSoup as BS
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import win10toast
import dotenv
from dotenv import dotenv_values
import winsound

toast = win10toast.ToastNotifier()
config = dotenv.dotenv_values('config.env')

courses = config['courses']
sections = config['sections']
if len(courses) == 0 and len(sections) == 0:
    assert "courses and sections in config.env is empty, add some course names or section IDs."

term = config['term']
if len(term) == 0:
    assert "term in config.env is missing. Format is FA22, WI22, SP22, S122, S222"

useSubjects = 'tabs-crs' if (config['useSubjects'] == "True") else 'tabs-sec'

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Origin': 'https://act.ucsd.edu',
    'Pragma': 'no-cache',
    'Referer': 'https://act.ucsd.edu/scheduleOfClasses/scheduleOfClassesStudentResult.htm',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = [
    ('selectedTerm', term),
    ('xsoc_term', ''),
    ('loggedIn', 'false'),
    ('tabNum', useSubjects),
    ('_selectedSubjects', '1'),
    ('schedOption1', 'true'),
    ('_schedOption1', 'on'),
    ('_schedOption11', 'on'),
    ('_schedOption12', 'on'),
    ('schedOption2', 'true'),
    ('_schedOption2', 'on'),
    ('_schedOption4', 'on'),
    ('_schedOption5', 'on'),
    ('_schedOption3', 'on'),
    ('_schedOption7', 'on'),
    ('_schedOption8', 'on'),
    ('_schedOption13', 'on'),
    ('_schedOption10', 'on'),
    ('_schedOption9', 'on'),
    ('schDay', 'M'),
    ('schDay', 'T'),
    ('schDay', 'W'),
    ('schDay', 'R'),
    ('schDay', 'F'),
    ('schDay', 'S'),
    ('_schDay', 'on'),
    ('_schDay', 'on'),
    ('_schDay', 'on'),
    ('_schDay', 'on'),
    ('_schDay', 'on'),
    ('_schDay', 'on'),
    ('schStartTime', '12:00'),
    ('schStartAmPm', '0'),
    ('schEndTime', '12:00'),
    ('schEndAmPm', '0'),
    ('_selectedDepartments', '1'),
    ('schedOption1Dept', 'true'),
    ('_schedOption1Dept', 'on'),
    ('_schedOption11Dept', 'on'),
    ('_schedOption12Dept', 'on'),
    ('schedOption2Dept', 'true'),
    ('_schedOption2Dept', 'on'),
    ('_schedOption4Dept', 'on'),
    ('_schedOption5Dept', 'on'),
    ('_schedOption3Dept', 'on'),
    ('_schedOption7Dept', 'on'),
    ('_schedOption8Dept', 'on'),
    ('_schedOption13Dept', 'on'),
    ('_schedOption10Dept', 'on'),
    ('_schedOption9Dept', 'on'),
    ('schDayDept', 'M'),
    ('schDayDept', 'T'),
    ('schDayDept', 'W'),
    ('schDayDept', 'R'),
    ('schDayDept', 'F'),
    ('schDayDept', 'S'),
    ('_schDayDept', 'on'),
    ('_schDayDept', 'on'),
    ('_schDayDept', 'on'),
    ('_schDayDept', 'on'),
    ('_schDayDept', 'on'),
    ('_schDayDept', 'on'),
    ('schStartTimeDept', '12:00'),
    ('schStartAmPmDept', '0'),
    ('schEndTimeDept', '12:00'),
    ('schEndAmPmDept', '0'),
    ('courses', courses),
    ('sections', sections),
    ('instructorType', 'begin'),
    ('instructor', ''),
    ('titleType', 'contain'),
    ('title', ''),
    ('_hideFullSec', 'on'),
    ('_showPopup', 'on'),
]

server = None

for i in range(100000):
    timestamp = time.strftime('%a %H:%M:%S')
    print(timestamp)

    # Get HTML response from link with selected classes inputted
    url = 'https://act.ucsd.edu/scheduleOfClasses/scheduleOfClassesStudentResult.htm'
    try:
        response = requests.post(url, headers=headers, data=data, timeout=10)
    except requests.exceptions.ReadTimeout:
        print("Connection timed out")
        continue
    except ConnectionResetError:
        print("Connection was unexpectedly closed. Waiting 180 seconds")
        sleep(180)
        continue
    soup = BS(response.text, 'html.parser')

    # Find all sections for classes using specific class tag
    section_all = soup.find_all('tr', {'class': 'sectxt'})
    for section in section_all:
        col = section.find_all('td')
        try:
            resource_elem = col[10]
        except IndexError:
            # Skip if not found, such as when section is cancelled
            continue

        # Check if the class is not full
        resource_text = resource_elem.getText().strip()
        if "FULL" not in resource_text and len(resource_text) > 0:
            print("Open class found:")
            toast.show_toast("Open class found: {courses}",duration=5)

            # Setup message contents
            section_id = col[2].getText().strip()
            day = col[5].getText().strip()
            time_ = col[6].getText().strip()
            building = col[7].getText().strip()
            prof = col[9].getText().strip()
            # Get 4 letter class ID using an excessively long regex
            class_id = re.search('subject=....', col[12].find('span')[
                                     'onclick'])[0][-4:]

            message = f'Open class {class_id}, ID {section_id}, {day}, {time_} with {prof}'
            print(message)
            bb()
            
                

                
            

    delay = 0.5
    print(f"Checking again in {delay} seconds\n")
    time.sleep(delay)


Thu 14:08:17
Checking again in 0.5 seconds

Thu 14:08:18
Checking again in 0.5 seconds

Thu 14:08:19
Checking again in 0.5 seconds

Thu 14:08:19
Checking again in 0.5 seconds

Thu 14:08:20
Checking again in 0.5 seconds

Thu 14:08:22
Checking again in 0.5 seconds

Thu 14:08:22
Checking again in 0.5 seconds

Thu 14:08:23
Checking again in 0.5 seconds

Thu 14:08:24
Checking again in 0.5 seconds

Thu 14:08:25
Checking again in 0.5 seconds

Thu 14:08:26
Checking again in 0.5 seconds

Thu 14:08:27
Checking again in 0.5 seconds

Thu 14:08:28
Checking again in 0.5 seconds

Thu 14:08:29
Checking again in 0.5 seconds

Thu 14:08:29
Checking again in 0.5 seconds

Thu 14:08:30
Checking again in 0.5 seconds

Thu 14:08:31
Checking again in 0.5 seconds

Thu 14:08:32
Checking again in 0.5 seconds

Thu 14:08:33
Checking again in 0.5 seconds

Thu 14:08:34
Checking again in 0.5 seconds

Thu 14:08:35
Checking again in 0.5 seconds

Thu 14:08:35
Checking again in 0.5 seconds

Thu 14:08:36
Checking again in 0

In [3]:
import winsound

In [14]:
bb()

KeyboardInterrupt: 

In [13]:
def bb():
    for _ in range(120):
        winsound.Beep(500,500)
        time.sleep(0.7)
        winsound.Beep(700,500)
        time.sleep(0.2)
        winsound.Beep(700,500)